In [1]:
import numpy as np
import tensorflow as tf

l2 regularization

In [2]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

x_train = xy[:, :-1]
y_train = xy[:, [-1]] 

In [3]:
def normalization(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data,0) - np.min(data,0)
    return numerator/denominator

xy = normalization(xy)
xy

array([[1.        , 1.        , 0.        , 1.        , 1.        ],
       [0.70548491, 0.70439552, 1.        , 0.71881783, 0.83755792],
       [0.54412549, 0.50274824, 0.57608696, 0.60646801, 0.6606331 ],
       [0.33890353, 0.31368023, 0.10869565, 0.45989134, 0.43800918],
       [0.51436   , 0.4258239 , 0.30434783, 0.58504805, 0.42624401],
       [0.49556179, 0.4258239 , 0.31521739, 0.48131134, 0.49276137],
       [0.11436064, 0.        , 0.20652174, 0.22007776, 0.18597238],
       [0.        , 0.07747099, 0.5326087 , 0.        , 0.        ]])

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

W = tf.Variable(tf.random.normal([4,1]), dtype=tf.float32)
b = tf.Variable(tf.random.normal([1]),dtype=tf.float32)

def linearReg_fn(features):
    hypothesis = tf.matmul(features, W)+b
    return hypothesis

def l2_loss(loss, beta=0.01):
    W_reg = tf.nn.l2_loss(W)
    loss = tf.reduce_mean(loss+W_reg*beta)
    return loss

def loss_fn(hypothesis, labels, flag =False):
    cost = tf.reduce_mean(tf.square(hypothesis-labels))
    if(flag):
        cost = l2_loss(cost)
    return cost


In [5]:
is_decay = True
starter_learning_rate = 0.1

if(is_decay):
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = starter_learning_rate,
                                                                   decay_steps = 50, 
                                                                   decay_rate = 0.96, 
                                                                   staircase=True)
    # decay_steps : 곱할 횟수 정의(50번에 마다 적용)
    # 50회마다 0.96을 곱하는
    # initial_learning_rate * decay_rate ^ (step / decay_steps)
    optimizer = tf.keras.optimizers.SGD(learning_rate)
else:
    optimizer = tf.keras.optimizers.SGD(learning_rate = starter_learning_rate)

def grad(features, labels, l2_flag):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(linearReg_fn(features), labels, l2_flag)
    return tape.gradient(loss_value, [W,b]), loss_value

EPOCHS = 101

for step in range(EPOCHS):
    for features, labels in dataset:
        features = tf.cast(features, tf.float32)
        labels = tf.cast(labels, tf.float32)
        grads, loss_value = grad(features, labels, False) 
        optimizer.apply_gradients(grads_and_vars = zip(grads,[W,b]))
        if step % 10 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_value))

Iter: 0, Loss: 4460604555264.0000
Iter: 10, Loss: nan
Iter: 20, Loss: nan
Iter: 30, Loss: nan
Iter: 40, Loss: nan
Iter: 50, Loss: nan
Iter: 60, Loss: nan
Iter: 70, Loss: nan
Iter: 80, Loss: nan
Iter: 90, Loss: nan
Iter: 100, Loss: nan
